# 1501. Countries You Can Safely Invest In
[問題](https://leetcode.com/problems/countries-you-can-safely-invest-in/description/?envType=study-plan-v2&envId=premium-sql-50)

## 解答の方針
global_call_averageを算出する。  
phone_numberとreを使ってマッチングしてmergeする。

callsを平たくする。



In [1]:
import pandas as pd

In [182]:
data = [[3, 'Jonathan', '051-1234567'], [12, 'Elvis', '051-7654321'], [1, 'Moncef', '212-1234567'], [2, 'Maroua', '212-6523651'], [7, 'Meir', '972-1234567'], [9, 'Rachel', '972-0011100']]
person = pd.DataFrame(data, columns=['id', 'name', 'phone_number']).astype({'id':'Int64', 'name':'object', 'phone_number':'object'})
data = [['Peru', '051'], ['Israel', '972'], ['Morocco', '212'], ['Germany', '049'], ['Ethiopia', '251']]
country = pd.DataFrame(data, columns=['name', 'country_code']).astype({'name':'object', 'country_code':'object'})
data = [[1, 9, 33], [2, 9, 4], [1, 2, 59], [3, 12, 102], [3, 12, 330], [12, 3, 5], [7, 9, 13], [7, 1, 3], [9, 7, 1], [1, 7, 7]]
calls = pd.DataFrame(data, columns=['caller_id', 'callee_id', 'duration']).astype({'caller_id':'Int64', 'callee_id':'Int64', 'duration':'Int64'})

In [139]:
person

,id,name,phone_number
0,3,Jonathan,051-1234567
1,12,Elvis,051-7654321
2,1,Moncef,212-1234567
3,2,Maroua,212-6523651
4,7,Meir,972-1234567
5,9,Rachel,972-0011100


In [140]:
country

,name,country_code
0,Peru,051
1,Israel,972
2,Morocco,212
3,Germany,049
4,Ethiopia,251


In [141]:
calls

,caller_id,callee_id,duration
0,1,9,33
1,2,9,4
2,1,2,59
3,3,12,102
4,3,12,330
5,12,3,5
6,7,9,13
7,7,1,3
8,9,7,1
9,1,7,7


In [181]:
import re
def find_safe_countryies(person: pd.DataFrame, country: pd.DataFrame, calls: pd.DataFrame) -> pd.DataFrame:
    patern = r'(\d{3})-.*'
    person["country_code"] = person["phone_number"].apply(lambda x: re.match(pattern, x).groups()[0])
    person = person.merge(country, on="country_code", how="left")
    calls = pd.concat([
        calls[["caller_id", "duration"]].rename({"caller_id": "id"},axis=1),
        calls[["callee_id", "duration"]].rename({"callee_id": "id"}, axis=1)
    ])
    average_duration = calls["duration"].mean()
    merged = calls.merge(person, on="id", how="inner")
    merged = merged.rename({"name_y": "country"}, axis=1)
    grouped = merged.groupby("country")["duration"].mean().reset_index()
    result = grouped[grouped["duration"] > average_duration]["country"].reset_index()
    return result[["country"]]

In [183]:
find_safe_countryies(person, country, calls)

,country
0,Peru


## Test scripts

In [147]:
import re

In [148]:
pattern = r'(\d{3})-.*'

In [149]:
person["country_code"] = person["phone_number"].apply(lambda x: re.match(pattern, x).groups()[0])

In [150]:
person

,id,name,phone_number,country_code
0,3,Jonathan,051-1234567,051
1,12,Elvis,051-7654321,051
2,1,Moncef,212-1234567,212
3,2,Maroua,212-6523651,212
4,7,Meir,972-1234567,972
5,9,Rachel,972-0011100,972


In [151]:
person = person.merge(country, on="country_code", how="inner")

In [152]:
person

,id,name_x,phone_number,country_code,name_y
0,3,Jonathan,051-1234567,051,Peru
1,12,Elvis,051-7654321,051,Peru
2,1,Moncef,212-1234567,212,Morocco
3,2,Maroua,212-6523651,212,Morocco
4,7,Meir,972-1234567,972,Israel
5,9,Rachel,972-0011100,972,Israel


In [89]:
calls = pd.concat([calls[["caller_id", "duration"]].rename({"caller_id": "id"}, axis=1), 
calls[["callee_id", "duration"]].rename({"callee_id": "id"}, axis=1)])

In [90]:
calls

,id,duration
0,1,33
1,2,4
2,1,59
3,3,102
4,3,330
5,12,5
6,7,13
7,7,3
8,9,1
9,1,7


In [91]:
average_duration = calls["duration"].mean()

In [92]:
merged = calls.merge(person, on="id", how="left")

In [93]:
merged

,id,duration,name_x,phone_number,country_code,name_y
0,1,33,Moncef,212-1234567,212,Morocco
1,2,4,Maroua,212-6523651,212,Morocco
2,1,59,Moncef,212-1234567,212,Morocco
3,3,102,Jonathan,051-1234567,051,Peru
4,3,330,Jonathan,051-1234567,051,Peru
5,12,5,Elvis,051-7654321,051,Peru
6,7,13,Meir,972-1234567,972,Israel
7,7,3,Meir,972-1234567,972,Israel
8,9,1,Rachel,972-0011100,972,Israel
9,1,7,Moncef,212-1234567,212,Morocco


In [94]:
merged = merged.rename({"name_x": "name", "name_y": "country"}, axis=1)

In [95]:
merged

,id,duration,name,phone_number,country_code,country
0,1,33,Moncef,212-1234567,212,Morocco
1,2,4,Maroua,212-6523651,212,Morocco
2,1,59,Moncef,212-1234567,212,Morocco
3,3,102,Jonathan,051-1234567,051,Peru
4,3,330,Jonathan,051-1234567,051,Peru
5,12,5,Elvis,051-7654321,051,Peru
6,7,13,Meir,972-1234567,972,Israel
7,7,3,Meir,972-1234567,972,Israel
8,9,1,Rachel,972-0011100,972,Israel
9,1,7,Moncef,212-1234567,212,Morocco


In [96]:
grouped = merged.groupby("country")["duration"].mean().reset_index()

In [97]:
grouped

,country,duration
0,Israel,9.375
1,Morocco,27.5
2,Peru,145.666667


In [98]:
result = grouped[grouped["duration"] > average_duration]["country"].reset_index()

In [99]:
result[["country"]]

,country
0,Peru
